In [1]:
import cv2
import mediapipe as mp
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [2]:
cap = cv2.VideoCapture(0) #Checks for camera
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils
 
#To access speaker through the library pycaw 
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0
 
volMin,volMax = volume.GetVolumeRange()[:2]
 
while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    lmList = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb
 
        length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[30,350],[volMin,volMax]) 
        volbar=np.interp(length,[30,350],[400,150])
        volper=np.interp(length,[30,350],[0,100])
        
        
        print(vol,int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 98),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord(' '): #By using spacebar delay will stop
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

-57.89904459567726 157
-48.21752030775691 189
-47.692932373048436 191
-49.26455346908935 185
-49.7397068411685 184
-48.82224639592645 187
-47.587022372986084 191
-47.30008665517773 192
-47.38672028082415 192
-47.843897963559485 190
-46.89595539035169 193
-48.37703646046068 188
-51.923734117781045 176
-52.0060380797963 176
-45.769940064187 197
-43.50000000000001 204
-54.08831175456858 169
-59.83131615820537 150
-78.49113355875058 88
-80.07110110734934 83
-80.97001872659905 80
-84.97401687806564 66
-84.19375093872036 69
-84.7572235105952 67
-86.7 61
-86.61168849513365 61
-86.1596178382709 62
-83.52955519790054 71
-79.88227717327862 83
-76.16096395508337 96
-59.03729772957208 153
-56.025516848056476 163
-46.37014412434565 195
-44.952019184655335 200
-37.733069045778514 224
-35.290818395278805 232
-25.848878719249953 263
-25.848878719249953 263
-23.755369408187946 270
-22.303325338923074 275
-21.063238089618935 279
-21.3844512067283 278
-21.46198470157435 278
-21.778248035744895 277
-20.57

-15.501396658942213 298
-12.45936027884828 308
-24.759112666920245 267
-27.689586729858974 257
-27.536072911322137 258
-25.635965324335984 264
-29.32741579673653 252
-25.798863644515805 264
-29.759784689303302 250
-29.59549085101078 251
-29.26559566484991 252
-33.893038315506686 237
-33.834839984722855 237
-35.27339101892305 232
-34.43619908196555 235
-58.70950421522793 154
-75.22585013808119 99
-94.98700843903282 33
-91.44677160230817 45
-40.298686257541895 215
-44.533066887760874 201
-34.85322530579185 233
-20.84050855666962 280
-23.653518822262498 271
-23.297490858603368 272
-23.242981952617654 272
-24.55411756963568 268
-24.915731881973215 266
-31.955082312319647 243
-22.87174663978297 273
-18.37009754132238 288
-21.659913606956238 277
-22.173373846328857 276
-20.589574103668923 281
-21.61025242873079 277
-40.1375301117819 216
-24.431705988025286 268
-21.877800799064516 277
-67.93330335732627 123
-53.52223781087605 171
-37.75381051687762 224
-22.729409872056948 274
-75.506780440243